### 1 - reconcile_accounts

In [1]:
# Imports
import csv
import pandas as pd
from pathlib import Path
from pprint import pprint
from core_logic_classes import Transaction
from core_logic_classes import TransactionReconciler

In [ ]:
# 1 - Create Example CSV Files for Transaction Reconciliation

# Data for transactions1.csv
transactions1_data = [
    ['2020-12-04', 'Tecnologia', '16.00', 'Bitbucket'],
    ['2020-12-04', 'Jurídico', '60.00', 'LinkSquares'],
    ['2020-12-05', 'Tecnologia', '50.00', 'AWS'],
]

# Data for transactions2.csv
transactions2_data = [
    ['2020-12-04', 'Tecnologia', '16.00', 'Bitbucket'],
    ['2020-12-05', 'Tecnologia', '49.99', 'AWS'],
    ['2020-12-04', 'Jurídico', '60.00', 'LinkSquares'],
]

# Save both transaction groups to CSV
with open('transactions1.csv', 'w', newline='', encoding='utf-8') as file1:
    writer = csv.writer(file1)
    writer.writerows(transactions1_data)

with open('transactions2.csv', 'w', newline='', encoding='utf-8') as file2:
    writer = csv.writer(file2)
    writer.writerows(transactions2_data)

print("CSV files created successfully!")


In [2]:
# 2 - Reconcile Transactions from Two CSV Files
from typing import List, Tuple

def reconcile_accounts(data1: List[List[str]], data2: List[List[str]]) -> Tuple[List[List[str]], List[List[str]]]:
    """
    Reconciles two groups of transactions represented as lists of lists.

    Each transaction is converted into a Transaction object and then passed
    to the TransactionReconciler to determine matches based on department,
    value, beneficiary, and ±1 day date tolerance.

    Args:
        data1 (List[List[str]]): Raw transaction data from the first file.
        data2 (List[List[str]]): Raw transaction data from the second file.

    Returns:
        Tuple[List[List[str]], List[List[str]]]: Two lists of transactions with
        an additional column indicating 'FOUND' or 'MISSING'.
    """

    # Convert each row to a Transaction object
    group_a = [Transaction.from_list(row) for row in data1]
    group_b = [Transaction.from_list(row) for row in data2]

    # Reconcile using the helper class
    reconciler = TransactionReconciler(group_a, group_b)
    return reconciler.reconcile()


In [ ]:
# 3 - Load Transaction CSVs, Reconcile, and Display Results

# Read CSV files into DataFrames
transactions1_df = Transaction.read_transaction_csv("transactions1.csv")
transactions2_df = Transaction.read_transaction_csv("transactions2.csv")

# Convert DataFrames to list of lists (raw data format)
transactions1_data = transactions1_df.values.tolist()
transactions2_data = transactions2_df.values.tolist()

# Reconcile transactions using the core function
out1, out2 = reconcile_accounts(transactions1_data, transactions2_data)

# Print results
from pprint import pprint
pprint(out1)
pprint(out2)

In [ ]:
# 4 - Optional: Saving the reconciled transactions to CSV files
TransactionReconciler.save_reconcile_csv(out1, "transactions1_reconciled.csv", out2, "transactions2_reconciled.csv")

### 2 - last_lines

In [ ]:
# 0 - Optional Creating a sample file to test the last_lines function
file_content = """This is a file
This is line 2
And this is line 3
"""

with open("my_file.txt", "w", encoding="utf-8") as f:
    f.write(file_content)

print("File 'my_file.txt' created successfully!")

In [3]:
# Importing the txt_lines class from core_logic_classes
from core_logic_classes import txt_lines

# It is possible to change the path of the txt file here.
str_path = "my_file.txt"
txt_file_obj = txt_lines(str_path)

def last_lines(lines):
    """
    Prints each line from the given list after stripping trailing whitespace.

    Args:
        lines (list[str]): List of lines to print.
    """
    for line in lines:
        print(line.strip())


In [ ]:
# 1 - Calling the read_lines method: read the lines in txt file and return a list of strings.
txt_file_obj.read_lines()

['This is a file\n', 'This is line 2\n', 'And this is line 3\n']

In [ ]:
# 2 - Calling reverse_lines_chunked method: read list of lines in txt and return a list of strings in reverse order.
lines = txt_file_obj.reverse_lines_chunked()
lines

['And this is line 3\n', 'This is line 2\n', 'This is a file\n']

In [ ]:
# 3 - Calling the last_lines function: print the content of the txt in reverse order.
last_lines(lines)

And this is line 3
This is line 2
This is a file


In [7]:
# 4 - Using the iterator to read lines one by one. -> Run this cell multiple times to see the iterator in action.
txt_file_obj.__next__()

'And this is line 3\n'

In [ ]:
# 5 - Using reset method to reset the iterator.
txt_file_obj.reset()

### 3 - computed_property

In [9]:
from core_logic_classes import computed_property # Function which has the decorator to create a computed property and call the wrapper function.

In [10]:
class Vector:
    def __init__(self, x, y, z, color=None):
        """
        Initializes a 3D vector with optional color.

        Args:
            x (float): X-coordinate
            y (float): Y-coordinate
            z (float): Z-coordinate
            color (str, optional): Optional color label
        """
        self.x, self.y, self.z = x, y, z
        self.color = color

    @computed_property('x', 'y', 'z')
    def magnitude(self):
        """
        Returns the Euclidean norm of the vector.
        This property is cached and only recalculated if x, y, or z changes.
        """
        print("computing magnitude")
        return (self.x**2 + self.y**2 + self.z**2) ** 0.5
    
v = Vector(3, 4, 0)

In [ ]:
# Printing the method for the first time -> It should compute the magnitude and print "computing magnitude".
v.magnitude

computing magnitude


5.0

In [ ]:
# Printing the method for the second time -> It should not compute the magnitude again, returning the cached value.
v.magnitude

5.0

In [13]:
# Changing the value of the vector to test the decorator and the cache storaged value.
v = Vector(9,2,6)
v.magnitude

computing magnitude


11.0

In [14]:
# Changing the value of a parameter that is not in the decorator.
v.color = 'red'
v.magnitude

11.0

In [ ]:
# Changing the value of a parameter that is in the decorator.
v.y = 18
v.magnitude

computing magnitude


21.0

In [16]:
# Check if cached value remains when dependencies don't change
v.y = 18
v.magnitude

21.0

In [ ]:
class Circle:
    def __init__(self, radius=1):
        """
        Initializes a circle with a given radius.
        
        Args:
            radius (float): Radius of the circle. Default is 1.
        """
        self.radius = radius

    @computed_property('radius','area') # 'area' is safely ignored if missing
    def diameter(self):
        """
        Computes the diameter of the circle.
        Cached as long as radius remains unchanged.
        """
        print('computing diameter')
        return self.radius * 2

    @diameter.setter
    def diameter(self, d):
        """
        Updates the radius based on a new diameter.
        """
        self.radius = d / 2

    @diameter.deleter
    def diameter(self):
        """
        Resets the radius to zero.
        """
        self.radius = 0

circle = Circle()

print("🔹 Initial diameter access:")
print(circle.diameter)

print("\n🔹 Accessing diameter again (should use cache):")
print(circle.diameter)

print("\n🔹 Changing diameter via setter to 10:")
circle.diameter = 10
print(f"New radius: {circle.radius}")

print("\n🔹 Deleting diameter (resets radius):")
del circle.diameter
print(f"Radius after deletion: {circle.radius}")

🔹 Initial diameter access:
computing diameter
2

🔹 Accessing diameter again (should use cache):
2

🔹 Changing diameter via setter to 10:
New radius: 5.0
computing diameter
Recomputed diameter: 10.0

🔹 Deleting diameter (resets radius):
Radius after deletion: 0


In [20]:
circle = Circle()
circle.diameter

computing diameter


2

In [21]:
# Checking if the computed_property is respecting the cache value when a MISSING parameter is passed ('area' in this case).
circle.diameter

2

In [22]:
# Checking if the setter method trrigers the computed property when the computed property has a unused parameter ('area')
circle.radius = 2
circle.diameter

computing diameter


4

In [23]:
# Checking if the setter method trrigers the computed property when the computed property has a unused parameter ('area')
circle.radius = 2
circle.diameter

4

In [24]:
# Checking the docstring treatment of the computed_property
help(Circle)

Help on class Circle in module __main__:

class Circle(builtins.object)
 |  Circle(radius=1)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, radius=1)
 |      Initializes a circle with a given radius.
 |      
 |      Args:
 |          radius (float): Radius of the circle. Default is 1.
 |  
 |  __setattr__ = custom_setattr(self, name, value)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  diameter
 |      Computes the diameter of the circle.
 |      Cached as long as radius remains unchanged.

